# Calculate Ridge Metrics

This notebook contains the code required to calculate ridge metrics from the previously created datasets:
- Ridges
- Migration Pathways
- Packets
- Ridge Area Raster
- DEM

From these datasets, we will calculate ridge width, amplitude, and migration distance at every ridge-transect intersection and aggregate/summarize these measurements at the intersection, ridge, transect, packet, and bend scales. 


In [ ]:
from pathlib import Path

import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt

from scrollstats import calculate_ridge_metrics, RasterClipper

## Set input/output paths

In [ ]:
# Vector Datasets
ridge_path = Path("example_data/output/LBR_025_ridges_manual_smoothed.geojson")
transect_path = Path("example_data/output/LBR_025_transects.geojson")
packet_path = Path("example_data/input/LBR_025_packets.geojson")
centerline_path = Path("example_data/input/LBR_025_cl.geojson") # only needed for plotting

# Raster Datasets
bin_clip_path = Path("example_data/output/LBR_025_agr_clip_denoise500px80p.tif")
dem_clip_path = Path("example_data/input/LBR_025_dem_clip.tif")

# Output
output_dir = Path("example_data/output")


## Read in datasets from input paths

In [ ]:
# Vector Data
ridges = gpd.read_file(ridge_path)
transects = gpd.read_file(transect_path)
packets = gpd.read_file(packet_path)
cl = gpd.read_file(centerline_path)

# Raster Data
bin_raster = rasterio.open(bin_clip_path)
dem = rasterio.open(dem_clip_path)

## Calculate the ridge metrics

The `calculate_ridge_metrics` function takes the transects, ridges, clipped ridge area (binary) and DEM rasters as input and returns two GeoDataFrames
- `rich transects`
- `itx points`

The `rich_transects` retain the same geometry as the input transects but have the following additional fields:
- `r_000` ... `r_###`: the distance between the channel centerline and that ridge along the specified transect
- `dem_signal`: dem values sampled along the transect with 1m sampling 
- `bin_signal`: binary raster values sampled along the transect with 1m sampling
- `clean_bin_signal`:
- `ridge_count_raster`: the ridge count along the transect calcualted from the number of unique ridges in `bin_signal`
- `fft_spacing`: the fft-derived dominant wavelength from `bin_signal` - corresponds to a representative spacing between ridges along the transect

The `itx` points are the collection of point intersections between the ridges and transects. `itx` contains the following fields:
- `start_distances`
- `transect_position`: vertex position of the intersection point along the transect
- `metric_confidence`: a simple metric derived from `bin_signal` describing the confidence in other ridge metrics.
- `relative_vertex_distances`: the relative along-transect distance of the substring that corresponds to the intersection. A relative vertex position of 0.5 is at the exact midpoint of the transect.
- `vertex_indices`: the indicies of the `dem_signal` or `binary_signal` of the transect that correspond to the location of the itx substring
- `dem_signal`: the section of the transect's `dem_signal` that corresponds to the itx substring
- `dem_signal_selection`: ukn
- `bin_signal`: the section of the transect's `bin_signal` that corresponds to the itx substring
- `bool_mask`: ukn
- `pre_mig_dist`: the migration distance from the previous ridge 
- `post_mig_dist`: the migration distance to the next ridge
- `pre_mig_time`: the difference in `deposit_year` between the intersected and previous ridge
- `post_mig_time`: the difference in `deposit_year` between the intersected and next ridge
- `pre_mig_rate`: `pre_mig_dist` / `pre_mig_time`
- `post_mig_rate`: `post_mig_dist` / `post_mig_time`
- `ridge_width`: the width of the ridge derived from `bin_signal` measured in px
- `ridge_amp`: the amplitude of the ridge derived from `dem_signal` measured in units of the DEM
- `deposit_year`: `deposit_year` of the corresponding ridge
- `substring_geometry`: the `LineString` of the substring corresponding to the intersection
- `geometry`: the `Point` geometry of the intersection
- `swale_dq_adjustment`: ukn



In [ ]:
rich_transects, itx = calculate_ridge_metrics(transects, bin_raster, dem, ridges)
itx = itx.loc["LBR_025"]

In [ ]:
# Add packets
itx_w_packets = itx.sjoin(packets.drop("bend_id", axis=1))
itx_w_packets = itx_w_packets.reset_index().set_index(["transect_id", "ridge_id", "packet_id"])
ridge_metrics_w_packets = itx_w_packets[["ridge_amp", "ridge_width", "pre_mig_dist"]]
ridge_metrics_w_packets.columns = ridge_metrics_w_packets.columns.rename("metrics")

In [ ]:
itx_w_packets.head()

In [ ]:
# Plot itx
fig, ax = plt.subplots(1,1, figsize=(10,6))
ridges.plot(ax=ax, color="k", ls="--", lw=0.5, zorder=0)
transects.plot(ax=ax, color="k", lw=1, zorder=1)
cl.plot(ax=ax, color="tab:blue", lw=5, zorder=2)

itx_w_packets.plot(column="ridge_amp", ax=ax, zorder=2, legend=True, 
                   legend_kwds={"label": "Ridge Amplitude [m]"})

ax.set_title("Ridge amplitude at each intersection")
